# Webカメラキャプチャ処理プログラム
- USB Webカメラ画像をキャプチャし、表示回路２でディスプレイに表示する
- Webカメラの解像度と表示回路２の対応解像度に注意すること
    - Webカメラ解像度 :  
    　　VGA　( 640 x  480)  
    　　HD　 (1280 x  720)
    - 表示回路２の対応解像度 :  
    　　VGA　( 640 x  480)  
    　　XGA　(1024 x  768)  
    　　SXGA (1280 x 1024) 

### [STEP-1] 定数定義
- 表示解像度パラメータ（VGA or XGA）
- レジスタアドレス設定

In [ ]:
# 表示回路２解像度パラメータ
PIXEL_VGA_WIDTH  = 640
PIXEL_VGA_HEIGHT = 480

PIXEL_XGA_WIDTH  = 1024
PIXEL_XGA_HEIGHT = 768

# 表示回路レジスタオフセット
RA_DISPADDR = 0x00000000
RA_DISPCTRL = 0x00000004
RA_DISPINT  = 0x00000008
RA_DISPFIFO = 0x0000000c

#GPIOレジスタ
RA_GPIO_DATA = 0x00000000

### [STEP-2] DisplayPort設定用クラス定義
- Zynq内部の専用レジスタの定義とレジスタ設定

In [ ]:
import time
from pynq import MMIO
from pynq.lib.video import *

class LiveDisplayPort(DisplayPort):
    
    # DisplayPortレジスタパラメータ
    __DP_REG_BASE_ADDR__                    = 0xfd4a0000
    __DP_REG_ADDR_RANGE__                   = 0x0000cc20
    __RA_DP_MAIN_STREAM_ENABLE__            = 0x00000084
    __RA_V_BLEND_SET_GLOVAL_ALPHA_REG__     = 0x0000a00c
    __RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__ = 0x0000b070
    __RA_AV_BUF_AUD_VID_CLK_SOURCE__        = 0x0000b120
    __RA_AV_BUF_SRST_REG__                  = 0x0000b124
    
    def __init__(self, event_loop=None):
        super().__init__(event_loop)
        
    def configure(self, mode, pixelformat):
        super().configure(mode, pixelformat)
        self.__setup_live_video__()
        
    def __setup_live_video__(self):
        dpreg = MMIO(self.__DP_REG_BASE_ADDR__, self.__DP_REG_ADDR_RANGE__)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000000)
        dpreg.write(self.__RA_V_BLEND_SET_GLOVAL_ALPHA_REG__, 0x00000000)
        dpreg.write(self.__RA_AV_BUF_OUTPUT_AUDIO_VIDEO_SELECT__, 0x00000050)
        dpreg.write(self.__RA_AV_BUF_AUD_VID_CLK_SOURCE__, 0x00000002)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000002)
        time.sleep(1)
        dpreg.write(self.__RA_AV_BUF_SRST_REG__, 0x00000000)
        time.sleep(1)
        dpreg.write(self.__RA_DP_MAIN_STREAM_ENABLE__, 0x00000001)

### [STEP-3] Overlay
- 表示回路２のコンフィギュレーション

In [ ]:
from pynq import Overlay
pl = Overlay("design_1.bit")

### [STEP-4] 表示回路２の解像度設定　（※繰り返す場合はここから実行）
- RESOLの値を変更してパターン表示回路の解像度を設定
    - 0 : VGA設定
    - 1 : XGA設定
    - 2 : SXGA設定
- 表示回路２では、RESOLの値は0,1のみ対応

In [ ]:
RESOL = 0
pl.axi_gpio_0.write(RA_GPIO_DATA,(RESOL&1))
pl.axi_gpio_0.read(RA_GPIO_DATA)

### [STEP-5] DisplayPort初期化
- 表示回路２の解像度に設定を合わせる
    - VGA設定  : dp.configure(VideoMode(640,480,24),PIXEL_RGB)
    - XGA設定  : dp.configure(VideoMode(1024,768,24),PIXEL_RGB)
    - SXGA設定 : dp.configure(VideoMode(1280,1024,24),PIXEL_RGB)

In [ ]:
from pynq.lib.video import *
lvdp = LiveDisplayPort()

if (RESOL & 1) == 1 :
    PIXEL_WIDTH  = PIXEL_XGA_WIDTH
    PIXEL_HEIGHT = PIXEL_XGA_HEIGHT
    print("XGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)
else :
    PIXEL_WIDTH  = PIXEL_VGA_WIDTH
    PIXEL_HEIGHT = PIXEL_VGA_HEIGHT
    print("VGA setting @ DisplayPort", PIXEL_WIDTH, PIXEL_HEIGHT)

lvdp.configure(VideoMode(PIXEL_WIDTH, PIXEL_HEIGHT, 24), PIXEL_RGB)

### [STEP-6] VRAM領域確保【修正対象】
- 必要なVRAM領域の設定はユーザ側で検討してプログラムを修正する
- 高さ x 幅 x 4byte(RGBA)で１フレーム分のデータとなる

In [ ]:
from pynq import allocate

fb0 = allocate(shape = (PIXEL_HEIGHT, PIXEL_WIDTH, 4), dtype = "u1")

### [STEP-7] Webカメラ設定【修正対象】
- 解像度はVGAまたはHD

In [ ]:
import cv2

CAM_PIXEL_WIDTH = 640
CAM_PIXEL_HEIGHT = 480

capture = cv2.VideoCapture(0)

capture.set(3, CAM_PIXEL_WIDTH)
capture.set(4, CAM_PIXEL_HEIGHT)

### [STEP-8] 表示ONのレジスタ設定
- 表示ON @ DISPCTRL

In [ ]:
pl.regbus_0.write(RA_DISPCTRL, 0x00000001)

### [STEP-9] Webカメラキャプチャ画像表示【実装対象】
- Webカメラ画像キャプチャ
- フォーマット変換(BGR2BGRA)
- VRAMへの書き込み
- 同期処理（VBLANK待ち)
- 表示開始先頭アドレス設定 @ DISPADDR
- 指定フレーム数の繰り返し処理
- FPSの計算と結果の表示

In [ ]:
#ここにユーザがプログラムを記述










### [STEP-10] 表示OFF＆PYNQの初期画面に戻す
- 表示OFF @ DISPCTRL
- DisplayPortの制御は、PL側からPS側に切り替わります。

In [ ]:
#リリース処理
capture.release()
fb0.freebuffer()

#表示OFF設定
pl.regbus_0.write(RA_DISPCTRL, 0x00000000)
lvdp.close()